In [1]:
import h2o
import time
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

import os 
from os.path import join, split

from io import StringIO
import pandas as pd

data_path = (join(
    split(os.getcwd())[0],'data'))

In [2]:
h2o.init(url='http://192.168.1.100:54321')

Checking whether there is an H2O instance running at http://192.168.1.100:54321. connected.


H2O cluster uptime:,12 mins 04 secs
H2O cluster version:,3.10.5.4
H2O cluster version age:,"21 days, 8 hours and 33 minutes"
H2O cluster name:,kuba
H2O cluster total nodes:,1
H2O cluster free memory:,7.102 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://192.168.1.100:54321
H2O connection proxy:,None


In [23]:
filename = join(data_path, join('ml-1m', 'ratings.dat'))
pd_ratings_frame = pd.read_csv(filename, sep='%', names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
pd_ratings_frame.describe()

,UserID,MovieID,Rating,Timestamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [38]:
pd_movies_frame = pd.read_csv(join(data_path, 'ml-1m', 'movies.dat'), sep='%', names=['MovieID', 'Title', 'Genres'], encoding='latin-1')

pd_all_frame = pd_ratings_frame.merge(pd_movies_frame[['MovieID', 'Title']], on='MovieID')
pd_all_frame.head()

,UserID,MovieID,Rating,Timestamp,Title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975)
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975)
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975)
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975)


In [85]:
pd_ratings_frame = pd_all_frame.pivot(index='UserID', columns='Title', values='Rating')
pd_ratings_frame.head()

Title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
UserID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
ratings_frame = (h2o.H2OFrame(pd_ratings_frame,
                  column_types=list(map(lambda x: 'float', range(pd_ratings_frame.shape[1]))),
                  na_strings=['nan']))

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [90]:
%time reconstructed = glrm.reconstruct(ratings_frame)

CPU times: user 552 ms, sys: 12 ms, total: 564 ms
Wall time: 6.17 s


## Your ratings

In [91]:
my_ratings = list(zip(*[
  ['Matrix, The (1999)', 5],
  ['Godfather, The (1972)', 4],
  ['Fear and Loathing in Las Vegas (1998)', 5],
  ['Trainspotting (1996)', 4],
  ['Clockwork Orange, A (1971)', 5],
  ['Full Metal Jacket (1987)', 5],
  ['Total Recall (1990)', 4],
  ['Alien (1979)', 5],
  ['GoodFellas (1990)', 4]]))

my_ratings_df = pd.DataFrame(data={'UserId': [0 for __ in range(len(my_ratings[0]))], 'Title': my_ratings[0], 'Rating': my_ratings[1]})
my_ratings_df

,Rating,Title,UserId
0,5,"Matrix, The (1999)",0
1,4,"Godfather, The (1972)",0
2,5,Fear and Loathing in Las Vegas (1998),0
3,4,Trainspotting (1996),0
4,5,"Clockwork Orange, A (1971)",0
5,5,Full Metal Jacket (1987),0
6,4,Total Recall (1990),0
7,5,Alien (1979),0
8,4,GoodFellas (1990),0


In [105]:
pivoted_df = (my_ratings_df
              .pivot(index='UserId', columns='Title', values='Rating')
              )

Title
Alien (1979)                             int64
Clockwork Orange, A (1971)               int64
Fear and Loathing in Las Vegas (1998)    int64
Full Metal Jacket (1987)                 int64
Godfather, The (1972)                    int64
GoodFellas (1990)                        int64
Matrix, The (1999)                       int64
Total Recall (1990)                      int64
Trainspotting (1996)                     int64
dtype: object

In [115]:
full_my_df = pivoted_df.reindex(columns=pd_ratings_frame.columns)
full_my_df.shape

(1, 3706)

## Your recommendations H2O Frame

In [187]:
pd_full_frame = pd.concat([full_my_df, pd_ratings_frame], axis=0)

full_h2o_frame = h2o.H2OFrame(pd_full_frame,
                            header=-1,
                            column_types=['numeric' for __ in  range(pd_full_frame.shape[1])],
                            na_strings=['nan'])

Parse progress: |█████████████████████████████████████████████████████████| 100%


## Training GLRM model

In [166]:
from h2o.estimators import H2OGeneralizedLowRankEstimator

glrm = H2OGeneralizedLowRankEstimator(
  k=10,
  regularization_x='L1',
  regularization_y='L1',
  transform='Demean',
  max_iterations=100)

%time glrm.train(training_frame=full_frame)

In [182]:
%time reconstructed_frame = glrm.reconstruct(full_frame)

(6041, 3706)